In [1]:
import pandas
import datetime
import numpy
import scipy.optimize
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
import bs4
import urllib
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
import sys
sys.path.insert(0, '../py')
import get_saps_vels
import saps_lshell_vel_map
from davitpy.pydarn.radar import *
from davitpy.pydarn.plotting import *
from davitpy import pydarn
from davitpy.utils import *
%matplotlib inline

In [2]:
# # Base directory where all the files are stored
# baseDir = "/home/bharat/Documents/code/new-vel-data/veldata/"
# # Fit results output go to
# outFitResFil = "../data/spwidth-all.csv"
# # POES Boundary data
# inpPOESFile = "../data/processedSaps.txt"
# # create a DF with the data
# sapsDataDF = pandas.read_csv(inpPOESFile,\
#              sep=' ', dtype={'dateStr':'str', 'time': 'str'})
# sapsDataDF["date"] = pandas.to_datetime( \
#                 sapsDataDF['dateStr'] + "-" +\
#                 sapsDataDF['time'], format='%Y%m%d-%H%M')
# failedDTObjs = []
# totCnt = 0
# # read data from a file
# for root, dirs, files in os.walk(baseDir):
#     for fNum, fName in enumerate(files):
#         currInpLosFile = root + fName    
#         # READ data
#         sapsObj = get_saps_vels.ProcessVels(currInpLosFile, sapsDataDF)
#         totCnt += sapsObj.velsDataDF.shape[0]
#         spwDF = sapsObj.velsDataDF[ ["date", "MLAT", "normMLT", "MLON", "spwdth", "radCode"] ]
#         with open(outFitResFil, 'a') as fra:
#             spwDF.to_csv(fra, header=False,\
#                               index=False, sep=' ' )
# print "totCnt-->", totCnt

In [8]:
# intensities, pcoll = overlayFan(scans, myMap, myFig, param, coords,
#                                         gsct=gsct, site=sites[i], fov=fovs[i],
#                                         fill=fill, velscl=velscl, dist=dist,
#                                         cmap=cmap, norm=norm)
rad = [ "inv" ]
sTime = datetime.datetime(2014,12,16,0,14)


tbands = []
tFreqBands = []
for i in range(len(rad)):
    if tFreqBands == [] or tFreqBands[i] == []:
        tbands.append([8000, 20000])
    else:
        tbands.append(tFreqBands[i])

myFiles = []
myBands = []
for i in range(len(rad)):
    f = radDataOpen(sTime, rad[i], sTime + datetime.timedelta(seconds=120))
    if(f is not None):
        myFiles.append(f)
        myBands.append(tbands[i])
        
sites = []        
allBeams = [''] * len(myFiles)        
# go through all open files
for i in range(len(myFiles)):
    # read until we reach start time
    allBeams[i] = radDataReadRec(myFiles[i])
    while (allBeams[i].time < sTime and allBeams[i] is not None):
        allBeams[i] = radDataReadRec(myFiles[i])

    # check that the file has data in the target interval
    if(allBeams[i] is None):
        myFiles[i].close()
        myFiles[i] = None
        continue
        
        
bndTime = sTime + datetime.timedelta(seconds=120)        
scans = []        
ft = 'None'
# go though all files
pcoll = None
for i in range(len(myFiles)):
    scans = []
    # check that we have good data at this time
    if(myFiles[i] is None or allBeams[i] is None): continue
    ft = allBeams[i].fType
    print allBeams[i]
    # until we reach the end of the time window
    while(allBeams[i] is not None and allBeams[i].time < bndTime):
        # filter on frequency
        if (allBeams[i].prm.tfreq >= myBands[i][0] and
                allBeams[i].prm.tfreq <= myBands[i][1]):
            scans.append(allBeams[i])
        # read the next record
        allBeams[i] = radDataReadRec(myFiles[i]) 
        
coords="geo"

Beam record FROM: 2014-12-16 00:14:00.035000
bmnum  = 0 
fPtr  = object 
fitex  = None 
fit  = object 
prm  = object 
recordDict  = object 
stid  = 64 
lmfit  = None 
exflg  = None 
iqflg  = None 
offset  = 1012600 
rawacf  = object 
lmflg  = None 
rawflg  = None 
fType  = fitex 
time  = 2014-12-16 00:14:00.035000 
acflg  = None 
cp  = 3503 
iqdat  = object 
fitacf  = None 
channel  = 0 



In [11]:
myData=scans
site = pydarn.radar.site(radId=myData[0].stid, dt=myData[0].time)
fov = pydarn.radar.radFov.fov(site=site, rsep=myData[0].prm.rsep,
                                      ngates=myData[0].prm.nrang + 1,
                                      nbeams=site.maxbeam, coords=coords,
                                      date_time=myData[0].time, model="IS")

In [ ]:

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1,1,1)
# myMap = utils.plotUtils.mapObj(boundinglat=30., gridLabels=True,\
#                         coords=coords, ax=ax, datetime=sTime)
# intensities, pcoll = overlayFan(scans, myMap, fig, 'velocity', coords)
fan.plotFan(sTime,['cly'],param='velocity')